In [ ]:

from typing import Callable, Tuple, TypeVar, cast, get_origin, get_args, Union, Any, Optional, Dict, List
from components import CallChannel, ComponentStarter

from components.two_wheels import TwoWheelsV3
from components.logger import LoggerComponent
from components.controller.bluetooth_controller import BlueToothCarControlSPP_V2
from components.controller.bluetooth_SPP_server import start_bluetooth_server_v2b
from components.gyroscope.gyroscope import AngularSpeedControlV3
from components.camera import Picamera2V2
from data_collection.data_collection import LoggerSet
from components import PitchAngularVelocityController, get_switches
from multiprocessing import Manager
import numpy as np
import datetime
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)

# from components import numpy_sample_setup
# r, w = numpy_sample_setup(['d', 'd'], [0, 0])

%load_ext autoreload
%autoreload 2

In [ ]:
loggerset = LoggerSet('../log/testsession'+str(datetime.datetime.now()), overwrite_ok=False)
#loggerset = LoggerSet('../log/session'+str(datetime.datetime.now()), overwrite_ok=False)
manager = Manager()

In [ ]:

logger_process = ComponentStarter(
    LoggerComponent, 
    manager, 
    init_kwargs=dict(
        loggerset = loggerset
    ),
    loop_intervals={'step': 1/100},
)

two_wheel_process = ComponentStarter(
    TwoWheelsV3, 
    manager, 
    init_kwargs=dict(
        left_pin = 33, 
        right_pin = 32, 
        dir_pins = (16, 18), 
        name='TwoWheelsV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = TwoWheelsV3.entry

)

angular_speed_control_process = ComponentStarter(
    AngularSpeedControlV3, 
    manager, 
    init_kwargs=dict(
        i2c_address=0x68, 
        bus_num=1,
        name='AngularSpeedControlV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = AngularSpeedControlV3.entry
)

bluetooth_control_process = ComponentStarter(
    BlueToothCarControlSPP_V2, 
    manager, 
    init_kwargs={},
    loop_intervals={'step': 1/100},
)

bt_ser_out, bt_ser_process_man = start_bluetooth_server_v2b(manager)


picamera_process = ComponentStarter(
    Picamera2V2, 
    manager, 
    init_kwargs=dict(
        resolution=(114, 64), 
        framerate=30,
        name='Picamera2V2'
    ),
    loop_intervals={'step': 1/30},
    sample_setup_kwargs=dict(default_values=[np.zeros((64, 114, 3), dtype=np.uint8)])
)

pitch_process = ComponentStarter(
    PitchAngularVelocityController,
    manager, 
    init_kwargs=dict(speed=100),
    loop_intervals={'step': 1/30}, 
    instantiator=PitchAngularVelocityController.entry
    

)


In [ ]:
two_wheel_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)

angular_speed_control_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)


bluetooth_control_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)
pitch_process.register_outgoing_rpc(
    dict(logger=logger_process.incoming_rpcs['log'])
)

picamera_process.register_outgoing_rpc(
    dict(
        log=logger_process.incoming_rpcs['log'],
        setup_video_saver=logger_process.incoming_rpcs['setup_video_saver'],
        save_video_frame=logger_process.incoming_rpcs['save_video_frame']
        )
)

def get_pitch_drive_switch():
    return bool(bt_ser_out[0]().get('start'))

to_imu_controller = get_switches(pitch_process.outgoing_samples, bluetooth_control_process.outgoing_samples, get_pitch_drive_switch)


two_wheel_process.register_incoming_samples(
    angular_speed_control_process.outgoing_sample_readers
)

angular_speed_control_process.register_incoming_samples(
    to_imu_controller
)


bluetooth_control_process.register_incoming_samples(
    bt_ser_out
)


In [ ]:
bt_ser_process_man.start()
bluetooth_control_process.start()
logger_process.start()
two_wheel_process.start()
angular_speed_control_process.start()
picamera_process.start()
pitch_process.start()


In [ ]:
bt_ser_process_man.kill()
bluetooth_control_process.process_starter.kill()
#logger_process.process_starter.kill()
two_wheel_process.process_starter.kill()
angular_speed_control_process.process_starter.kill()
picamera_process.process_starter.kill()
pitch_process.process_starter.kill()



In [ ]:
from utils import mon_samples
mon_samples(pitch_process.outgoing_samples)

# playground

In [ ]:
from data_collection.data_collection import Logger

In [ ]:
sesspath = '../log/testsession2024-07-06 17:18:55.153292/'

In [ ]:
l = Logger(sesspath,name='PitchAngularVelocityController', overwrite_ok=True)
lw2 = Logger(sesspath,name='TwoWheelsV3', overwrite_ok=True)

In [ ]:
df = l.load_as_df()
dfw2 = lw2.load_as_df()
df['power_norm'] = df['power']/1e4

In [ ]:
import pandas as pd
px.line(pd.concat([df, dfw2], axis=0), 'time',['pitch', 'speed', 'left','power_norm',])

In [ ]:
df[df.time > pd.to_datetime("2024-07-06 17:19:17")]

In [ ]:
px.line(df, 'time','speed')

In [ ]:
df['power_norm'] = df['power']/1e4

In [ ]:
px.line(df, 'time', 'power_norm')

In [ ]:
px.line(dfw2, 'time', 'left')

In [ ]:
px.imshow(np.log(np.stack(v.tolist())))

In [ ]:
from scipy.fft import fft, fftfreq
v = df['sig'].dropna()
v = v.apply(fft)
v = v.apply(lambda x: x.real**2+x.imag**2)
v = v.apply(lambda x: x[:len(x)//2])
freqs = fftfreq((4000), 1/16000)[:2000]

In [ ]:
px.line(x=freqs[0:],y=np.log(v.iloc[590])[0:])

In [ ]:
px.line(x=freqs[0:],y=np.log(v.iloc[157])[0:])

In [ ]:
freqs

In [ ]:
v.iloc[0]

In [ ]:
import plotly.express as px
px.imshow(picamera_starter.outgoing_samples[0]())